# Day 24: 転移学習

## Learning Objectives
- 転移学習の概念を理解する
- 事前学習済みモデルを利用する
- Fine-tuning（微調整）を実装する
- 画像分類モデルをカスタマイズする

## 1. 転移学習の概要

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image
import cv2

# GPUが利用可能か確認
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

### なぜ転移学習が必要なのか？

1. **データ効率**: 少ないデータでも高精度なモデルが構築可能
2. **計算効率**: 事前学習済みモデルをベースにするため、学習時間が短縮
3. **性能向上**: 大規模データで学習されたモデルの知識を活用
4. **実用性**: カスタムタスクに迅速に適応

一般的な事前学習モデル：
- ResNet
- VGG16/19
- MobileNet
- EfficientNet
- Vision Transformer (ViT)

In [ ]:
def load_pretrained_model(model_name='MobileNetV2', input_shape=(224, 224, 3)):
    """事前学習済みモデルをロード"""
    base_model = tf.keras.applications.MobileNetV2(
        input_shape=input_shape,
        include_top=False,  # 分類層を除く
        weights='imagenet'
    )
    
    # ベースモデルの重みを凍結（学習させない）
    base_model.trainable = False
    
    print(f"{model_name}をロードしました")
    print(f"モデルの層の数: {len(base_model.layers)}")
    print(f"出力形状: {base_model.output_shape}")
    
    return base_model

## 2. Fine-tuningの実装

In [ ]:
def create_transfer_learning_model(base_model, num_classes=2):
    """転移学習用のモデルを構築"""
    # モデルの構築
    inputs = keras.Input(shape=(224, 224, 3))
    
    # 前処理（ImageNetの統計値）
    x = tf.keras.applications.mobilenet_v2.preprocess_input(inputs)
    
    # ベースモデル
    x = base_model(x, training=False)
    
    # Global Average Pooling
    x = layers.GlobalAveragePooling2D()(x)
    
    # Dropout（過学習防止）
    x = layers.Dropout(0.2)(x)
    
    # 出力層
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    # モデルの作成
    model = keras.Model(inputs, outputs)
    
    return model